THOMAS MCDONNELL QUANT TRADING STRATEGIES HOMEWORK 3

In [24]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import plotnine as p9
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm


import warnings

# Suppress the warning
warnings.filterwarnings("ignore")

In [33]:
FR = quandl.get_table("ZACKS/FR", api_key = 'uoxc6V3s61czWuHoGLcs', paginate=True)
FR['per_end_date'] = pd.to_datetime(FR['per_end_date'])
start_date = datetime(2015, 1, 1)
FR = FR[FR['per_end_date'] >= start_date]

In [ ]:
unique_tickers = FR['ticker'].unique().tolist()
x = dict()
for ticker in tqdm(unique_tickers):
    df = FR[FR['ticker'] == ticker]
    x[ticker] = df

In [ ]:
l = []
for key in x:
    if len(x[key]) > 39:
        l.append(key)

In [ ]:
eod_price = dict()
for ticker in tqdm(l):
    df =  quandl.get_table('QUOTEMEDIA/PRICES', ticker = [ticker], api_key = 'uoxc6V3s61czWuHoGLcs',
                    qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
                    date = { 'gte': '2015-01-01', 'lte': '2022-01-01' }, 
                     paginate=True)
    eod_price[ticker]  = df



In [65]:
eod_data = {key: value for key, value in eod_price.items() if len(value) == 1763}

In [ ]:
final_tickers = list(eod_data.keys())


In [ ]:
FR_data = dict()
for ticker in tqdm(final_tickers):
    df = FR[FR['ticker'] == ticker]
    FR_data[ticker] = df


In [74]:
FR_data

{'TJX':      m_ticker ticker          comp_name              comp_name_2 exchange  \
 None                                                                        
 216        ZY    TJX  The TJX Companies  The TJX Companies, Inc.     NYSE   
 217        ZY    TJX  The TJX Companies  The TJX Companies, Inc.     NYSE   
 218        ZY    TJX  The TJX Companies  The TJX Companies, Inc.     NYSE   
 219        ZY    TJX  The TJX Companies  The TJX Companies, Inc.     NYSE   
 220        ZY    TJX  The TJX Companies  The TJX Companies, Inc.     NYSE   
 221        ZY    TJX  The TJX Companies  The TJX Companies, Inc.     NYSE   
 222        ZY    TJX  The TJX Companies  The TJX Companies, Inc.     NYSE   
 223        ZY    TJX  The TJX Companies  The TJX Companies, Inc.     NYSE   
 224        ZY    TJX  The TJX Companies  The TJX Companies, Inc.     NYSE   
 225        ZY    TJX  The TJX Companies  The TJX Companies, Inc.     NYSE   
 226        ZY    TJX  The TJX Companies  The TJX Compani